In [1]:
import torch
from model import Model, args
from utils import *
# args.device = "cpu"

In [2]:
model = Model(args).to(args.device)

In [5]:
from transformers import AutoTokenizer,PreTrainedTokenizerFast

tokenizer = AutoTokenizer.from_pretrained("akshatshaw/LLaMA_hin1")

ImportError: tokenizers>=0.14,<0.19 is required for a normal functioning of this module, but found tokenizers==0.19.1.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [ ]:
start_context = "आपके द्वारा चुनी गई भाषा में लिखना आसान बनाता है|"
text_to_token_ids(start_context, tokenizer)

tensor([[ 6591,  6410, 17978,  6295,  7096,  6078, 12466,  8058, 10779,  6073,
            56]], device='cuda:0')

In [ ]:
torch.manual_seed(10)
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=args.max_seq_len
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 आपके द्वारा चुनी गई भाषा में लिखना आसान बनाता है | बाँ पागल दोबारा डिपार्ट मीटर 57 संजीव स्ट्राइक ]। सबा


In [ ]:
file_path = "/content/Full_text_Bible.txt"
with open(file_path, "r", encoding="utf-8") as file:
 text_data = file.read()

In [ ]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 3693013
Tokens: 936185


In [ ]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

In [ ]:
dataset = ModelDataset(text_data, tokenizer, args.max_seq_len, args.max_seq_len)
dataset.__len__()

3656

In [ ]:
torch.manual_seed(123)
train_loader = create_dataloader_v1(
 train_data,
 tokenizer,
 batch_size=2,
 max_length=args.max_seq_len,
 stride=args.max_seq_len,
 drop_last=True,
 shuffle=True,
 num_workers=0
)
val_loader = create_dataloader_v1(
 val_data,
 tokenizer,
 batch_size=2,
 max_length=args.max_seq_len,
 stride=args.max_seq_len,
 drop_last=False,
 shuffle=False,
 num_workers=0
)

In [ ]:
# print("Train loader:")
# for x, y in train_loader:
#  print(x.shape, y.shape)
# print("\nValidation loader:")
# for x, y in val_loader:
#  print(x.shape, y.shape)

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device= args.device):
 input_batch = input_batch.to(device)
 target_batch = target_batch.to(device)
 logits = model(input_batch)
 loss = torch.nn.functional.cross_entropy(
    logits.flatten(0, 1), target_batch.flatten()
 )
 return loss

In [ ]:
def calc_loss_loader(data_loader, model, device= args.device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch, target_batch, model, device
            )
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches # Averages the loss over all batches

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
torch.manual_seed(10)
with torch.no_grad():
  train_loss = calc_loss_loader(train_loader, model)
  val_loss = calc_loss_loader(val_loader, model)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
